In [1]:
import json
import shutil

from helper_functions import *
from utils import *

In [2]:
ROOT_PATH = 'datasets'
CONVERTER_PATH = os.path.join(ROOT_PATH, 'bhairavi_test/converter')
CLEANER_PATH = os.path.join(ROOT_PATH, 'bhairavi_test/cleaner')
LAST_BAR_MODE = 'remove'
test_per = 0.8

1. dividing the set into train and test

In [3]:
# MIDI/pop/pop_test ~ bhairavi_test
# MIDI/pop/pop_midi ~ bhairavi
l = [f for f in os.listdir(os.path.join(ROOT_PATH, 'bhairavi'))]
print(l)
idx = np.random.choice(len(l), int(test_per * len(l)), replace=False)
print(len(idx))
path_exists(os.path.join(ROOT_PATH, 'bhairavi_test/origin_midi'))
for i in idx:
    shutil.move(os.path.join(ROOT_PATH, 'bhairavi', l[i]),
                os.path.join(ROOT_PATH, 'bhairavi_test/origin_midi', l[i]))

['bhairavi27.wav.mid', 'Bhairavi03.wav.mid', 'Bhairavi02.wav.mid', 'bhairavi28.wav.mid', 'Bhairavi04.wav.mid', 'Bhairavi01.midi', 'Bhairavi05.wav.mid']
5


------convert clean-------------

In [4]:
midi_paths = get_midi_path(os.path.join(ROOT_PATH, 'bhairavi_test/origin_midi'))
midi_dict = {}
print(midi_paths)
print()
kv_pairs = [convert_midi_to_pianoroll(midi_path) for midi_path in midi_paths]
for kv_pair in kv_pairs:
    if kv_pair is not None:
        midi_dict[kv_pair[0]] = kv_pair[1]
print(kv_pairs)
with open(os.path.join(ROOT_PATH, 'bhairavi_test/midis.json'), 'w') as outfile:
    json.dump(midi_dict, outfile)

print("[Done] {} files out of {} have been successfully converted".format(len(midi_dict), len(midi_paths)))

with open(os.path.join(ROOT_PATH, 'bhairavi_test/midis.json')) as infile:
    midi_dict = json.load(infile)
    
count = 0
path_exists(CLEANER_PATH)
midi_dict_clean = {}
for key in midi_dict:
    if midi_filter(midi_dict[key]):
        midi_dict_clean[key] = midi_dict[key]
        count += 1
        shutil.copyfile(os.path.join(CONVERTER_PATH, key + '.npz'),
                            os.path.join(CLEANER_PATH, key + '.npz'))

print(midi_dict_clean)

with open(os.path.join(ROOT_PATH, 'bhairavi_test/midis_clean.json'), 'w') as outfile:
    json.dump(midi_dict_clean, outfile)

print("[Done] {} files out of {} have been successfully cleaned".format(count, len(midi_dict)))

['datasets/bhairavi_test/origin_midi/bhairavi27.wav.mid', 'datasets/bhairavi_test/origin_midi/Bhairavi03.wav.mid', 'datasets/bhairavi_test/origin_midi/Bhairavi02.wav.mid', 'datasets/bhairavi_test/origin_midi/Bhairavi04.wav.mid', 'datasets/bhairavi_test/origin_midi/Bhairavi01.midi']

bhairavi27.wav
{'first_beat_time': 0.023255813953488372, 'num_time_signature_change': 0, 'time_signature': None, 'tempo': 120.0}
Multitrack(name=None, resolution=24, tempo=array(shape=(2472, 1), dtype=float64), tracks=[Track(name='Bass', program=32, is_drum=False, pianoroll=array(shape=(0, 128), dtype=float64)), Track(name='Drums', program=0, is_drum=True, pianoroll=array(shape=(0, 128), dtype=float64)), Track(name='Guitar', program=24, is_drum=False, pianoroll=array(shape=(0, 128), dtype=float64)), Track(name='Piano', program=0, is_drum=False, pianoroll=array(shape=(2472, 128), dtype=uint8)), Track(name='Strings', program=48, is_drum=False, pianoroll=array(shape=(0, 128), dtype=float64))])
Bhairavi03.wav
{

------- choose the clean midi from the original sets ----------

In [5]:
if not os.path.exists(os.path.join(ROOT_PATH, 'bhairavi_test/cleaner_midi')):
    os.makedirs(os.path.join(ROOT_PATH, 'bhairavi_test/cleaner_midi'))
l = [f for f in os.listdir(os.path.join(ROOT_PATH, 'bhairavi_test/cleaner'))]
print(l)
print(len(l))
for i in l:
    shutil.copy(os.path.join(ROOT_PATH, 'bhairavi_test/origin_midi', os.path.splitext(i)[0] + '.mid'),
                os.path.join(ROOT_PATH, 'bhairavi_test/cleaner_midi', os.path.splitext(i)[0] + '.mid'))

['Bhairavi03.wav.npz', 'Bhairavi02.wav.npz']
2


--------- merge and crop --------------

In [6]:
if not os.path.exists(os.path.join(ROOT_PATH, 'bhairavi_test/cleaner_midi_gen')):
    os.makedirs(os.path.join(ROOT_PATH, 'bhairavi_test/cleaner_midi_gen'))
if not os.path.exists(os.path.join(ROOT_PATH, 'bhairavi_test/cleaner_npy')):
    os.makedirs(os.path.join(ROOT_PATH, 'bhairavi_test/cleaner_npy'))
l = [f for f in os.listdir(os.path.join(ROOT_PATH, 'bhairavi_test/cleaner_midi'))]
print(l)
count = 0
for i in range(len(l)):
    try:
        # multitrack = Multitrack(name=os.path.splitext(l[i])[0],resolution=4)
        x = pretty_midi.PrettyMIDI(os.path.join(ROOT_PATH, 'bhairavi_test/cleaner_midi', l[i]))
        multitrack = pypianoroll.from_pretty_midi(x)
        category_list = {'Piano': [], 'Drums': []}
        program_dict = {'Piano': 0, 'Drums': 0}
        for idx, track in enumerate(multitrack.tracks):
            if track.is_drum:
                category_list['Drums'].append(idx)
            else:
                category_list['Piano'].append(idx)
        tracks = []
        # merged = multitrack[category_list['Piano']].get_merged_pianoroll()
        merged = multitrack.blend()
        print(merged.shape)
        pr = get_bar_piano_roll(merged)
        print(pr.shape)
        pr_clip = pr[:, :, 24:108]
        print(pr_clip.shape)
        if int(pr_clip.shape[0] % 4) != 0:
            pr_clip = np.delete(pr_clip, np.s_[-int(pr_clip.shape[0] % 4):], axis=0)
        pr_re = pr_clip.reshape(-1, 64, 84, 1)
        print(pr_re.shape)
        save_midis(pr_re, os.path.join(ROOT_PATH, 'bhairavi_test/cleaner_midi_gen', os.path.splitext(l[i])[0] +
                                       '.mid'))
        np.save(os.path.join(ROOT_PATH, 'bhairavi_test/cleaner_npy', os.path.splitext(l[i])[0] + '.npy'), pr_re)
    except:
        count += 1
        print('Wrong', l[i])
        continue
print(count)

['Bhairavi03.wav.mid', 'Bhairavi02.wav.mid']
(12528, 128)
(195, 64, 128)
(195, 64, 84)
(192, 64, 84, 1)
(12984, 128)
(202, 64, 128)
(202, 64, 84)
(200, 64, 84, 1)
0


concatenate into a big binary numpy array file

In [7]:
l = [f for f in os.listdir(os.path.join(ROOT_PATH, 'bhairavi_test/cleaner_npy'))]
print(l)
train = np.load(os.path.join(ROOT_PATH, 'bhairavi_test/cleaner_npy', l[0]))
print(train.shape, np.max(train))
for i in range(1, len(l)):
    print(i, l[i])
    t = np.load(os.path.join(ROOT_PATH, 'bhairavi_test/cleaner_npy', l[i]))
    train = np.concatenate((train, t), axis=0)
print(train.shape)
np.save(os.path.join(ROOT_PATH, 'bhairavi_test/bhairavi_test_piano.npy'), (train > 0.0))

['Bhairavi03.wav.npy', 'Bhairavi02.wav.npy']
(192, 64, 84, 1) 105
1 Bhairavi02.wav.npy
(392, 64, 84, 1)


separate numpy array file into single phrases

In [8]:
if not os.path.exists(os.path.join(ROOT_PATH, 'bhairavi_test/phrase_test')):
    os.makedirs(os.path.join(ROOT_PATH, 'bhairavi_test/phrase_test'))
x = np.load(os.path.join(ROOT_PATH, 'bhairavi_test/bhairavi_test_piano.npy'))
print(x.shape)
count = 0
for i in range(x.shape[0]):
    if np.max(x[i]):
        count += 1
        np.save(os.path.join(ROOT_PATH, 'bhairavi_test/phrase_test/bhairavi_test_piano_{}.npy'.format(i+1)), x[i])
        print(x[i].shape)
   # if count == 11216:
   #     break
print(count)

(392, 64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)
(64, 84, 1)

other code

In [ ]:
# MIDI/pop/pop_test ~ bhairavi_test
# MIDI/pop/pop_midi ~ bhairavi

# filepaths = []
# msd_id_list = []
# for dirpath, _, filenames in os.walk(os.path.join(ROOT_PATH, 'MIDI/Sinfonie Data')):
#     for filename in filenames:
#         if filename.endswith('.mid'):
#             msd_id_list.append(filename)
#             filepaths.append(os.path.join(dirpath, filename))
# print(filepaths)
# print(msd_id_list)
# for i in range(len(filepaths)):
#     shutil.copy(filepaths[i], os.path.join(ROOT_PATH, 'MIDI/classic/classic_midi/{}'.format(msd_id_list[i])))
#
# x1 = np.load(os.path.join(ROOT_PATH, 'MIDI/classic/classic_train/classic_train_piano_1.npy'))
# x2 = np.load(os.path.join(ROOT_PATH, 'MIDI/classic/classic_train/classic_train_piano_2.npy'))
# x3 = np.load(os.path.join(ROOT_PATH, 'MIDI/classic/classic_train/classic_train_piano_3.npy'))
# x4 = np.load(os.path.join(ROOT_PATH, 'MIDI/classic/classic_train/classic_train_piano_4.npy'))
# x5 = np.load(os.path.join(ROOT_PATH, 'MIDI/classic/classic_train/classic_train_piano_5.npy'))
# x = np.concatenate((x1, x2, x3, x4, x5), axis=0)
# print(x.shape)
# np.save(os.path.join(ROOT_PATH, 'MIDI/classic/classic_train/classic_train_piano.npy'), x)
#
#
# multitrack = Multitrack(resolution=4, name='YMCA')
# x = pretty_midi.PrettyMIDI(os.path.join(ROOT_PATH, 'MIDI/famous_songs/P2C/origin/YMCA.mid'))
# multitrack.parse_pretty_midi(x)
#
# category_list = {'Piano': [], 'Drums': []}
# program_dict = {'Piano': 0, 'Drums': 0}
#
# for idx, track in enumerate(multitrack.tracks):
#     if track.is_drum:
#         category_list['Drums'].append(idx)
#     else:
#         category_list['Piano'].append(idx)
# tracks = []
# merged = multitrack[category_list['Piano']].get_merged_pianoroll()
#
# # merged = multitrack.get_merged_pianoroll()
# print(merged.shape)
#
# pr = get_bar_piano_roll(merged)
# print(pr.shape)
# pr_clip = pr[:, :, 24:108]
# print(pr_clip.shape)
# if int(pr_clip.shape[0] % 4) != 0:
#     pr_clip = np.delete(pr_clip, np.s_[-int(pr_clip.shape[0] % 4):], axis=0)
# pr_re = pr_clip.reshape(-1, 64, 84, 1)
# print(pr_re.shape)
# save_midis(pr_re, os.path.join(ROOT_PATH, 'MIDI/famous_songs/P2C/merged_midi/YMCA.mid'), 127)
# np.save(os.path.join(ROOT_PATH, 'MIDI/famous_songs/P2C/merged_npy/YMCA.npy'), (pr_re > 0.0))
#